In [106]:
#importing dependencies

import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import ipywidgets as widgets

#creating widget object
widgets.IntSlider()

IntSlider(value=0)

In [107]:
#storing path of data

file_path = "/Users/aryamannsrivastava/Desktop/IMPORTANT/CS661/Assignment2/mixture.vti"

In [108]:
#loading the .vti file using VTK and converting the 3D scalar volume into a NumPy array for visualization

reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(file_path)
reader.Update()

image_data = reader.GetOutput()
point_data = image_data.GetPointData().GetScalars()

scalars = vtk_to_numpy(point_data)
dims = image_data.GetDimensions()
volume = scalars.reshape(dims, order='F')
fs = volume.flatten()
dmin, dmax = fs.min(), fs.max()

X,Y,Z = np.mgrid[0:dims[0], 0:dims[1], 0:dims[2]]

In [109]:
#defining all the plotting functions

def plot_isosurface(val):
    fig = go.Figure(data=[
        go.Isosurface(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=fs,
            isomin=val,
            isomax=val,
            surface_count=1,
            colorscale='plasma',
            cmin=dmin,
            cmax=dmax,
            caps=dict(x_show=False, y_show=False, z_show=False),
            showscale=True
        )
    ])
    fig.update_layout(
        title=f"Isosurface at isovalue = {val:.2f}",
        scene=dict(aspectmode="data"),
        margin=dict(l=0, r=0, b=0, t=30)
    )
    return fig

def plot_histogram(val=None):
    if val is None:
        hdata = fs
        title = "Histogram of Entire Volume"
    else:
        lo, hi = val - 0.25, val + 0.25
        hdata = fs[(fs >= lo) & (fs <= hi)]
        title = f"Histogram for values in [{lo:.2f}, {hi:.2f}]"
    fig = go.Figure(data=[
        go.Histogram(x=hdata, nbinsx=40, marker_color='blue')
    ])
    fig.update_layout(
        title=title,
        xaxis_title="Scalar Value",
        yaxis_title="Frequency",
        margin=dict(l=0, r=0, b=0, t=30)
    )
    return fig

In [110]:
#creating the slider widget

slider = widgets.FloatSlider(
    value=0.0,
    min=dmin,
    max=dmax,
    step=0.01,
    description='Isovalue:',
    continuous_update=False,
    layout=widgets.Layout(width='95%')
)

In [111]:
#creating reset button and 2 output areas, one for isosurface and one for histogram plots

reset_btn = widgets.Button(description="Reset", button_style='warning')
out1 = widgets.Output()
out2 = widgets.Output()

In [ ]:
#defining the update and reset functions

def update(change=None):
    val = slider.value
    with out1:
        clear_output(wait=True)
        plot_isosurface(val).show()
    with out2:
        clear_output(wait=True)
        plot_histogram(val).show()

def reset(_):
    slider.value = 0.0

#connecting the slider and the reset button to their callbacks and laying out interface using horizontal blocks

slider.observe(update, names='value')
reset_btn.on_click(reset)

display(widgets.HBox([slider, reset_btn]))
display(widgets.HBox([out1, out2]))

#calling update() once initially to display the default plots(isovalue=0.0):

update()